In [ ]:
%run forestry.ipynb

time.sleep(0.000001)
cli.execute_command('q')

In [ ]:
pf = Princess(Genes(
        (BeeSpecies.FOREST, BeeSpecies.FOREST),
        (BeeFertility(2), BeeFertility(2))
     ))
dm = Drone(Genes(
        (BeeSpecies.MEADOWS, BeeSpecies.MEADOWS),
        (BeeFertility(2), BeeFertility(2))
))

In [ ]:
random.seed(2)
verbose = True
def printv(*args, **kwargs):
    global verbose
    if verbose:
        print(*args, **kwargs)

printv(pf, dm)
qc = pf.mate(dm)
pc, dc, dc2 = qc.die()
_, dc, dc2 = qc.die()
pc.inspect()
dc.inspect()
printv(pc)
printv(dc)

qc = pc.mate(dc)
pc, dc, dc2 = qc.die()
for i in range(3):
    _, _, dc = qc.die()
pc.inspect()
dc.inspect()
printv(pc)
printv(dc)
assert pc.genes == Genes(species=(BeeSpecies.COMMON, BeeSpecies.COMMON), fertility=(BeeFertility(2), BeeFertility(2)))

random.seed(40)
qcu = pc.mate(dm)
pcu, dcu, dcu2 = qcu.die()
pcu.inspect()
dcu.inspect()
qcu = pcu.mate(dcu)
pcu, _, dcu = qcu.die()
pcu.inspect()
dcu.inspect()
printv(pcu)
printv(dcu)
assert pcu.genes == Genes(species=(BeeSpecies.CULTIVATED, BeeSpecies.CULTIVATED), fertility=(BeeFertility(2), BeeFertility(2)))

random.seed(57)
qn = pcu.mate(dc)
pn, dn, _ = qn.die()
pn.inspect()
dn.inspect()
printv(pn)
printv(dn)

random.seed(17)
qn = pn.mate(dn)
pn, _, dn = qn.die()
pn.inspect()
dn.inspect()
printv(pn)
printv(dn)
assert pn.genes == Genes(species=(BeeSpecies.NOBLE, BeeSpecies.NOBLE), fertility=(BeeFertility(2), BeeFertility(2)))

In [ ]:
from collections import Counter
l = []
for i in range(10000):
    _, dc, dc2 = pf.mate(dm).die()
    dc.inspect()
    dc2.inspect()
    l.append(dc)
    l.append(dc2)
c = Counter(l)
s = sum(c.values())
for key in c:
    print(*key.genes.species, c[key] / s)

In [ ]:
from collections import Counter
l = []
princess = Princess(Genes(
        (BeeSpecies.COMMON, BeeSpecies.CULTIVATED),
        (BeeFertility(2), BeeFertility(2))
     ))
drone = Drone(Genes(
        (BeeSpecies.MEADOWS, BeeSpecies.MEADOWS),
        (BeeFertility(2), BeeFertility(2))
))
for i in range(10000):
    _, dc, dc2 = princess.mate(drone).die()
    dc.inspect()
    dc2.inspect()
    l.append(dc)
    l.append(dc2)
c = Counter(l)
s = sum(c.values())
for key in c:
    print(*key.genes.species, c[key] / s)

In [ ]:
inv = Inventory(100, render=printv)
inv.place_bees([pf, dm])
inv.render()
inv.mate(0, 1)
inv.render()
random.seed(0)
inv.place_bees(inv[0].take().die())
inv.render()
assert inv.empty_slots() == 97, inv.empty_slots()
assert inv.take(0).genes == Genes(species=(BeeSpecies.FOREST, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
assert inv.take(1).genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))
assert inv.take(2).genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))

In [ ]:
verbose = False
random.seed(0)
r = Resources(honey=0, render=printv)
api = Apiary(add_resources=r.add_resources, render=printv)
api.put_princess(pf)
assert api.princess.slot.genes == Genes(species=(BeeSpecies.FOREST, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))
api.render()

api.put_drone(dm)
api.render()
assert isinstance(api.princess.slot, Queen)
assert api.princess.slot.genes == Genes(species=(BeeSpecies.FOREST, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))

api.update()
assert r['honey'] == 1
api.render()

api.update()
api.update()
assert r['honey'] == 3
api.render()

api.update()
api.render()
r.render()
assert api.inv.empty_slots() == 4, inv.empty_slots()
assert api.inv.take(0).genes == Genes(species=(BeeSpecies.FOREST, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
assert api.inv.take(1).genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))
assert api.inv.take(2).genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.FOREST), fertility=(BeeFertility(2), BeeFertility(2)))

In [ ]:
try:
    verbose = True
    random.seed(0)

    sleep_time = 0.5

    cli = CLI()
    assert cli.resources['honey'] == 0
    assert len(cli.apiaries) == 1
    assert cli.inv.empty_slots() == 100
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 0\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: Slot empty\nDrone: Slot empty\n  ------ INV ------\n  Empty...\n'},)

    cli.execute_command('unshow')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 0\n------ INV ------\nEmpty...\n'},)

    cli.execute_command('unshow')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 0\n'},)

    cli.execute_command('unshow')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '\n'},)

    cli.execute_command('inv')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 0\n------ INV ------\nEmpty...\n'},)
    cli.execute_command('unshow')
    cli.execute_command('unshow')

    cli.execute_command('inv 0')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 0\nSlot empty\n'},)
    cli.execute_command('unshow')
    cli.execute_command('unshow')

    cli.execute_command('apiary 0')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 0\n------ APIARY ------\nPrincess: Slot empty\nDrone: Slot empty\n  ------ INV ------\n  Empty...\n'},)
    cli.execute_command('unshow')
    cli.execute_command('unshow')

    cli.execute_command('show resources')
    cli.execute_command('show inv')
    cli.execute_command('show apiary 0')
    cli.execute_command('forage')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 0\n------ INV ------\n0 MEADOWS Princess\n1 MEADOWS Drone\n------ APIARY ------\nPrincess: Slot empty\nDrone: Slot empty\n  ------ INV ------\n  Empty...\n'},)
    assert cli.inv[0].slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
    assert cli.inv[1].slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
    assert cli.inv.empty_slots() == 98

    cli.execute_command('put 0 0')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 0\n------ INV ------\n1 MEADOWS Drone\n------ APIARY ------\nPrincess: MEADOWS Princess\nDrone: Slot empty\n  ------ INV ------\n  Empty...\n'},)
    assert cli.inv[0].is_empty()
    assert cli.inv.empty_slots() == 99
    assert cli.apiaries[0].princess.slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))

    cli.execute_command('put 0 1')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    time.sleep(0.2) # UGLY: this wait should help a little with unstable updates
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 1\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: MEADOWS Queen, rem: 2\nDrone: Slot empty\n  ------ INV ------\n  Empty...\n'},)
    assert cli.inv[1].is_empty()
    assert cli.inv.empty_slots() == 100
    assert isinstance(cli.apiaries[0].princess.slot, Queen)
    assert cli.apiaries[0].princess.slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
    time.sleep(1)
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 2\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: MEADOWS Queen, rem: 1\nDrone: Slot empty\n  ------ INV ------\n  Empty...\n'},)
    time.sleep(1)
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 3\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: MEADOWS Queen, rem: 0\nDrone: Slot empty\n  ------ INV ------\n  Empty...\n'},)
    time.sleep(1)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 3\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: Slot empty\nDrone: Slot empty\n  ------ INV ------\n  0 MEADOWS Princess\n  1 MEADOWS Drone\n  2 MEADOWS Drone\n'},)
    assert cli.apiaries[0].inv[0].slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
    assert cli.apiaries[0].inv[1].slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
    assert cli.apiaries[0].inv[2].slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))

    cli.execute_command('reput 0 0')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 3\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: MEADOWS Princess\nDrone: Slot empty\n  ------ INV ------\n  1 MEADOWS Drone\n  2 MEADOWS Drone\n'},)
    
    assert cli.inv[0].is_empty()
    assert cli.inv.empty_slots() == 100
    assert cli.apiaries[0].princess.slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
    
    cli.execute_command('reput 0 1')
    time.sleep(sleep_time)
    printv(repr(cli.out.outputs))
    time.sleep(0.2) # UGLY: this wait should help a little with unstable updates
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 4\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: MEADOWS Queen, rem: 2\nDrone: Slot empty\n  ------ INV ------\n  2 MEADOWS Drone\n'},)
    assert cli.inv[1].is_empty()
    assert cli.inv.empty_slots() == 100
    assert isinstance(cli.apiaries[0].princess.slot, Queen)
    assert cli.apiaries[0].princess.slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
    time.sleep(1)
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 5\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: MEADOWS Queen, rem: 1\nDrone: Slot empty\n  ------ INV ------\n  2 MEADOWS Drone\n'},)
    time.sleep(1)
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 6\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: MEADOWS Queen, rem: 0\nDrone: Slot empty\n  ------ INV ------\n  2 MEADOWS Drone\n'},)
    time.sleep(1)
    printv(repr(cli.out.outputs))
    assert cli.out.outputs == ({'output_type': 'stream', 'name': 'stdout', 'text': '------ RESOURCES ------\nhoney: 6\n------ INV ------\nEmpty...\n------ APIARY ------\nPrincess: Slot empty\nDrone: Slot empty\n  ------ INV ------\n  0 MEADOWS Princess\n  1 MEADOWS Drone\n  2 MEADOWS Drone\n  3 MEADOWS Drone\n'},)
    assert cli.apiaries[0].inv[0].slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
    assert cli.apiaries[0].inv[1].slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))
    assert cli.apiaries[0].inv[2].slot.genes == Genes(species=(BeeSpecies.MEADOWS, BeeSpecies.MEADOWS), fertility=(BeeFertility(2), BeeFertility(2)))

    
    print('wow, all tests passed')
    
finally:
    cli.execute_command('q')